In [1]:
from keras.utils.data_utils import get_file
from keras.datasets import imdb
import pickle
import numpy as np
from keras.layers import (Input, Embedding, Dense, 
                          Flatten, Dropout, BatchNormalization, 
                          Convolution1D, MaxPooling1D, Merge)
from keras.preprocessing import sequence
from keras.models import Model, Sequential
from keras.optimizers import Adam
import os
from keras_utilities import load_array
import re
from numpy.random import normal

Using Theano backend.
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: Tesla K80 (CNMeM is enabled with initial size: 90.0% of memory, cuDNN 5103)


In [2]:
word_idx = imdb.get_word_index()

In [3]:
word_idx

{'fawn': 34701,
 'tsukino': 52006,
 'nunnery': 52007,
 'sonja': 16816,
 'vani': 63951,
 'woods': 1408,
 'spiders': 16115,
 'hanging': 2345,
 'woody': 2289,
 'trawling': 52008,
 "hold's": 52009,
 'comically': 11307,
 'localized': 40830,
 'disobeying': 30568,
 "'royale": 52010,
 "harpo's": 40831,
 'canet': 52011,
 'aileen': 19313,
 'acurately': 52012,
 "diplomat's": 52013,
 'rickman': 25242,
 'arranged': 6746,
 'rumbustious': 52014,
 'familiarness': 52015,
 "spider'": 52016,
 'hahahah': 68804,
 "wood'": 52017,
 'transvestism': 40833,
 "hangin'": 34702,
 'bringing': 2338,
 'seamier': 40834,
 'wooded': 34703,
 'bravora': 52018,
 'grueling': 16817,
 'wooden': 1636,
 'wednesday': 16818,
 "'prix": 52019,
 'altagracia': 34704,
 'circuitry': 52020,
 'crotch': 11585,
 'busybody': 57766,
 "tart'n'tangy": 52021,
 'burgade': 14129,
 'thrace': 52023,
 "tom's": 11038,
 'snuggles': 52025,
 'francesco': 29114,
 'complainers': 52027,
 'templarios': 52125,
 '272': 40835,
 '273': 52028,
 'zaniacs': 52130,

In [4]:
idx2word = {v: k for k, v in word_idx.iteritems()}

In [5]:
idx2word

{1: 'the',
 2: 'and',
 3: 'a',
 4: 'of',
 5: 'to',
 6: 'is',
 7: 'br',
 8: 'in',
 9: 'it',
 10: 'i',
 11: 'this',
 12: 'that',
 13: 'was',
 14: 'as',
 15: 'for',
 16: 'with',
 17: 'movie',
 18: 'but',
 19: 'film',
 20: 'on',
 21: 'not',
 22: 'you',
 23: 'are',
 24: 'his',
 25: 'have',
 26: 'he',
 27: 'be',
 28: 'one',
 29: 'all',
 30: 'at',
 31: 'by',
 32: 'an',
 33: 'they',
 34: 'who',
 35: 'so',
 36: 'from',
 37: 'like',
 38: 'her',
 39: 'or',
 40: 'just',
 41: 'about',
 42: "it's",
 43: 'out',
 44: 'has',
 45: 'if',
 46: 'some',
 47: 'there',
 48: 'what',
 49: 'good',
 50: 'more',
 51: 'when',
 52: 'very',
 53: 'up',
 54: 'no',
 55: 'time',
 56: 'she',
 57: 'even',
 58: 'my',
 59: 'would',
 60: 'which',
 61: 'only',
 62: 'story',
 63: 'really',
 64: 'see',
 65: 'their',
 66: 'had',
 67: 'can',
 68: 'were',
 69: 'me',
 70: 'well',
 71: 'than',
 72: 'we',
 73: 'much',
 74: 'been',
 75: 'bad',
 76: 'get',
 77: 'will',
 78: 'do',
 79: 'also',
 80: 'into',
 81: 'people',
 82: 'other',
 8

In [6]:
path = get_file('imdb_full.pkl', 
                origin='https://s3.amazonaws.com/text-datasets/imdb_full.pkl',
                md5_hash='d091312047c43cf9e4e38fef92437263')

In [7]:
f = open(path, 'rb')

In [8]:
(x_train, label_train), (x_test, label_test) = pickle.load(f)

In [9]:
x_train[:1]

[[23022,
  309,
  6,
  3,
  1069,
  209,
  9,
  2175,
  30,
  1,
  169,
  55,
  14,
  46,
  82,
  5869,
  41,
  393,
  110,
  138,
  14,
  5359,
  58,
  4477,
  150,
  8,
  1,
  5032,
  5948,
  482,
  69,
  5,
  261,
  12,
  23022,
  73935,
  2003,
  6,
  73,
  2436,
  5,
  632,
  71,
  6,
  5359,
  1,
  25279,
  5,
  2004,
  10471,
  1,
  5941,
  1534,
  34,
  67,
  64,
  205,
  140,
  65,
  1232,
  63526,
  21145,
  1,
  49265,
  4,
  1,
  223,
  901,
  29,
  3024,
  69,
  4,
  1,
  5863,
  10,
  694,
  2,
  65,
  1534,
  51,
  10,
  216,
  1,
  387,
  8,
  60,
  3,
  1472,
  3724,
  802,
  5,
  3521,
  177,
  1,
  393,
  10,
  1238,
  14030,
  30,
  309,
  3,
  353,
  344,
  2989,
  143,
  130,
  5,
  7804,
  28,
  4,
  126,
  5359,
  1472,
  2375,
  5,
  23022,
  309,
  10,
  532,
  12,
  108,
  1470,
  4,
  58,
  556,
  101,
  12,
  23022,
  309,
  6,
  227,
  4187,
  48,
  3,
  2237,
  12,
  9,
  215]]

In [10]:
# example of a training example
print(' '.join([idx2word[i] for i in x_train[1]]))

homelessness or houselessness as george carlin stated has been an issue for years but never a plan to help those on the street that were once considered human who did everything from going to school work or vote for the matter most people think of the homeless as just a lost cause while worrying about things such as racism the war on iraq pressuring kids to succeed technology the elections inflation or worrying if they'll be next to end up on the streets br br but what if you were given a bet to live on the streets for a month without the luxuries you once had from a home the entertainment sets a bathroom pictures on the wall a computer and everything you once treasure to see what it's like to be homeless that is goddard bolt's lesson br br mel brooks who directs who stars as bolt plays a rich man who has everything in the world until deciding to make a bet with a sissy rival jeffery tambor to see if he can live in the streets for thirty days without the luxuries if bolt succeeds he ca

In [11]:
x_train_w = [np.array([word if word < 4999 else 4999 for word in example]) for example in x_train]
x_train_w = sequence.pad_sequences(x_train_w, maxlen=500)

In [12]:
x_test_w = [np.array([word if word < 4999 else 4999 for word in example]) for example in x_test]
x_test_w = sequence.pad_sequences(x_test_w, maxlen=500)

In [13]:
inp = Input(shape=(x_train_w.shape[1],))
em = Embedding(input_dim=5000, output_dim=64, input_length=500)(inp)
fl = Flatten()(em)
d = Dense(100, activation='relu')(fl)
dr = Dropout(0.7)(d)
out = Dense(1, activation='sigmoid')(d)

mdl = Model(input=inp, output=out)

In [14]:
mdl.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
mdl.fit(x_train_w, label_train, batch_size=32, nb_epoch=2,
        validation_data=(x_test_w, label_test))

In [ ]:
mdl.fit(x_train_w, label_train, batch_size=32, nb_epoch=10,
        validation_data=(x_test_w, label_test))

In [ ]:
inp = Input(shape=(500,))
em = Embedding(5000, 32, input_length=500, dropout=0.2)(inp)
dr = Dropout(0.3)(em)
c1d = Convolution1D(64, 6, border_mode='same', activation='relu')(dr)
dr2 = Dropout(0.3)(c1d)
mp = MaxPooling1D()(dr2)
fl = Flatten()(mp)
de = Dense(100, activation='relu')(fl)
dr3 = Dropout(0.7)(de)
out = Dense(1, activation='sigmoid')(dr3)

mdl_conv = Model(input=inp, output=out)

mdl_conv.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

mdl_conv.fit(x_train_w, label_train, batch_size=32, nb_epoch=10,
             validation_data=(x_test_w, label_test))

### Get Glove Dataset

In [17]:
def get_glove_dataset(dataset):
    md5sums = {
        '6B.50d': '8e1557d1228decbda7db6dfd81cd9909',
        '6B.100d': 'c92dbbeacde2b0384a43014885a60b2c',
        '6B.200d': 'af271b46c04b0b2e41a84d8cd806178d',
        '6B.300d': '30290210376887dcc6d0a5a6374d8255'
    } 
    
    glove_path = os.path.abspath('data/glove/results')
    
    %mkdir -p glove_path
    
    return get_file(
        dataset,
        'http://files.fast.ai/models/glove/' + dataset + '.tgz',
        cache_subdir=glove_path,
        md5_hash=md5sums.get(dataset, None),
        untar=True
    ) 

In [18]:
def load_vectors(loc):
    return (load_array(loc+'.dat'), 
            pickle.load(open(loc+'_words.pkl', 'rb')),
            pickle.load(open(loc+'_idx.pkl', 'rb')))

In [19]:
vecs, words, wordidx = load_vectors(get_glove_dataset('6B.100d'))

Untaring file...


In [18]:
vecs[:2]

array([[  4.1800e-01,   2.4968e-01,  -4.1242e-01,   1.2170e-01,   3.4527e-01,  -4.4457e-02,
         -4.9688e-01,  -1.7862e-01,  -6.6023e-04,  -6.5660e-01,   2.7843e-01,  -1.4767e-01,
         -5.5677e-01,   1.4658e-01,  -9.5095e-03,   1.1658e-02,   1.0204e-01,  -1.2792e-01,
         -8.4430e-01,  -1.2181e-01,  -1.6801e-02,  -3.3279e-01,  -1.5520e-01,  -2.3131e-01,
         -1.9181e-01,  -1.8823e+00,  -7.6746e-01,   9.9051e-02,  -4.2125e-01,  -1.9526e-01,
          4.0071e+00,  -1.8594e-01,  -5.2287e-01,  -3.1681e-01,   5.9213e-04,   7.4449e-03,
          1.7778e-01,  -1.5897e-01,   1.2041e-02,  -5.4223e-02,  -2.9871e-01,  -1.5749e-01,
         -3.4758e-01,  -4.5637e-02,  -4.4251e-01,   1.8785e-01,   2.7849e-03,  -1.8411e-01,
         -1.1514e-01,  -7.8581e-01],
       [  1.3441e-02,   2.3682e-01,  -1.6899e-01,   4.0951e-01,   6.3812e-01,   4.7709e-01,
         -4.2852e-01,  -5.5641e-01,  -3.6400e-01,  -2.3938e-01,   1.3001e-01,  -6.3734e-02,
         -3.9575e-01,  -4.8162e-01,   2.329

In [19]:
words[:10]

['the', ',', '.', 'of', 'to', 'and', 'in', 'a', '"', "'s"]

In [20]:
wordidx

{'biennials': 130852,
 'verplank': 42458,
 'soestdijk': 274735,
 'woode': 311324,
 'mdbo': 212156,
 'sowell': 65544,
 'mdbu': 119490,
 'woods': 2507,
 'spiders': 19780,
 'mdbs': 285106,
 'mdbr': 131910,
 'woody': 10967,
 'trawling': 56782,
 'hwasung': 389348,
 'spidery': 126257,
 'regularize': 106404,
 'hennings': 85340,
 'canes': 34403,
 'canet': 110349,
 'caney': 97434,
 'yusaf': 223713,
 'chanthaburi': 154208,
 'igual': 120122,
 'hallucinate': 140964,
 'caned': 78728,
 'mirisch': 180868,
 'kalecik': 246349,
 'rickman': 54345,
 'jacquemod': 117004,
 'ioannidis': 142172,
 'canel': 299678,
 'canem': 357071,
 'afikoman': 354205,
 'dra\xc5\xbea': 228083,
 'heliothis': 265609,
 'replacer': 292996,
 'pigment': 29943,
 'bogyman': 343214,
 'transvestism': 191391,
 'nmu': 155319,
 '\xc4\x91\xc6\xb0\xe1\xbb\x9dng': 265245,
 '18f': 251230,
 'seamier': 168003,
 'illtyd': 307371,
 'wooded': 17054,
 'shipworms': 359396,
 'boorstein': 310641,
 'grueling': 17482,
 'persita': 353293,
 'wooden': 4836,

In [20]:
def create_emb():
    """
    1. create empty matrix to store pre-trained vectors
    2. grab each word in our vocabulary
    3. if the word has a pre-trained vector, grab it
    4. otherwise, set it to a random normal vector
    """
    n_factor = vecs.shape[1]
    emb = np.zeros((5000, n_factor))
    
    for i in range(1, len(emb)):
        # get the actual word
        word = idx2word[i]
        
        # make sure the word exists (i.e. isn't whitespace)
        # and matches the regex we deem reasonable
        if word and re.match(r"^[a-zA-Z0-9\-]*$", word):
            src_idx = wordidx[word]
            emb[i] = vecs[src_idx, :]
        else:
            emb[i] = normal(scale=0.6, size=(n_factor,))
        
    emb[-1] = normal(scale=0.6, size=(n_factor,))
    
    # why do we shrink these?
    emb /= 3
    
    return emb 

In [21]:
emb = create_emb()

In [28]:
mdl_pt = Sequential([
    Embedding(5000, 100, input_length=500, dropout=0.2, 
              weights=[emb], trainable=True),
    Dropout(0.25),
    Convolution1D(64, 5, border_mode='same', activation='relu'),
    Dropout(0.25),
    MaxPooling1D(),
    Flatten(),
    Dense(100, activation='relu'),
    Dropout(0.7),
    Dense(1, activation='sigmoid')])

mdl_pt.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])
mdl_pt.fit(x_train_w, label_train, batch_size=32, nb_epoch=10,
           validation_data=(x_test_w, label_test))

Train on 25000 samples, validate on 25000 samples
Epoch 1/10
25000/25000 [==============================] - 14s - loss: 0.4598 - acc: 0.7597 - val_loss: 0.2754 - val_acc: 0.8926
Epoch 2/10
25000/25000 [==============================] - 14s - loss: 0.2925 - acc: 0.8801 - val_loss: 0.2522 - val_acc: 0.8999
Epoch 3/10
25000/25000 [==============================] - 14s - loss: 0.2664 - acc: 0.8915 - val_loss: 0.2549 - val_acc: 0.9014
Epoch 4/10
25000/25000 [==============================] - 14s - loss: 0.2445 - acc: 0.9011 - val_loss: 0.2575 - val_acc: 0.8920
Epoch 5/10
25000/25000 [==============================] - 14s - loss: 0.2291 - acc: 0.9096 - val_loss: 0.2632 - val_acc: 0.8903
Epoch 6/10
25000/25000 [==============================] - 14s - loss: 0.2154 - acc: 0.9135 - val_loss: 0.2523 - val_acc: 0.8978
Epoch 7/10
25000/25000 [==============================] - 14s - loss: 0.2083 - acc: 0.9168 - val_loss: 0.2668 - val_acc: 0.8912
Epoch 8/10
25000/25000 [==============================

In [29]:
mdl_pt.layers[1].trainable = False

In [31]:
mdl_pt.compile(loss='binary_crossentropy', optimizer=Adam(1e-5), metrics=['accuracy'])
mdl_pt.fit(x_train_w, label_train, batch_size=32, nb_epoch=3,
           validation_data=(x_test_w, label_test))

Train on 25000 samples, validate on 25000 samples
Epoch 1/3
25000/25000 [==============================] - 14s - loss: 0.1518 - acc: 0.9418 - val_loss: 0.2505 - val_acc: 0.8978
Epoch 2/3
25000/25000 [==============================] - 14s - loss: 0.1485 - acc: 0.9424 - val_loss: 0.2498 - val_acc: 0.8979
Epoch 3/3
25000/25000 [==============================] - 14s - loss: 0.1520 - acc: 0.9419 - val_loss: 0.2494 - val_acc: 0.8979


In [ ]:
inp2 = Input(shape=(500,50))
convs = []
for i in range(3,7):
    x = Convolution1D(64, i, border_mode='same', activation='relu')(inp2)
    x = MaxPooling1D()(x)
    x = Flatten()(x)
    convs.append(x) 
out2 = Merge(mode="concat")(convs)
graph = Model(input=inp2, output=out2)

In [ ]:
inp = Input(shape=(500,))
ebd = Embedding(input_dim=emb.shape[0], output_dim=emb.shape[1], input_length=500, 
                dropout=0.2, weights=[emb], trainable=False)(inp)
dr = Dropout(0.25)(ebd) 
gr = graph(dr)
dr2 = Dropout(0.50)(gr)
de = Dense(100, activation='relu')(dr2)
dr3 = Dropout(0.50)(de)
out = Dense(1, activation='sigmoid')(dr3)

mdl_pt = Model(input=inp, output=out)

In [ ]:
mdl_pt.layers[1].trainable = True

In [ ]:
mdl_pt.compile(Adam(), loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
mdl_pt.fit(x_train_w, label_train, batch_size=32, nb_epoch=10,
           validation_data=(x_test_w, label_test))

In [ ]:
mdl_pt.fit(x_train_w, label_train, batch_size=32, nb_epoch=10,
           validation_data=(x_test_w, label_test))

### Add a little dropout

In [ ]:
inp = Input(shape=(500,))
ebd = Embedding(input_dim=emb.shape[0], output_dim=emb.shape[1], input_length=500, 
                dropout=0.2, weights=[emb], trainable=False)(inp)
dr = Dropout(0.25)(ebd) 
gr = graph(dr)
dr2 = Dropout(0.50)(gr)
de = Dense(100, activation='relu')(dr2)
dr3 = Dropout(0.70)(de)
out = Dense(1, activation='sigmoid')(dr3)

mdl_pt = Model(input=inp, output=out)

mdl_pt.compile(Adam(), loss='binary_crossentropy', metrics=['accuracy'])

mdl_pt.fit(x_train_w, label_train, batch_size=32, nb_epoch=10,
           validation_data=(x_test_w, label_test))

### Lower the dropout

In [ ]:
inp = Input(shape=(500,))
ebd = Embedding(input_dim=emb.shape[0], output_dim=emb.shape[1], input_length=500, 
                dropout=0.2, weights=[emb], trainable=False)(inp)
dr = Dropout(0.25)(ebd) 
gr = graph(dr)
dr2 = Dropout(0.20)(gr)
de = Dense(100, activation='relu')(dr2)
dr3 = Dropout(0.20)(de)
out = Dense(1, activation='sigmoid')(dr3)

mdl_pt = Model(input=inp, output=out)

mdl_pt.layers[1].trainable = True

mdl_pt.compile(Adam(), loss='binary_crossentropy', metrics=['accuracy'])

mdl_pt.fit(x_train_w, label_train, batch_size=32, nb_epoch=10,
           validation_data=(x_test_w, label_test))

Looking better. Let's try to lower the LR and continue to train.

In [ ]:
mdl_pt.compile(Adam(1e-3), loss='binary_crossentropy', metrics=['accuracy'])

mdl_pt.fit(x_train_w, label_train, batch_size=32, nb_epoch=10,
           validation_data=(x_test_w, label_test))

### No Dropout

In [ ]:
inp = Input(shape=(500,))
ebd = Embedding(input_dim=emb.shape[0], output_dim=emb.shape[1], input_length=500, 
                dropout=0.2, weights=[emb], trainable=False)(inp)
gr = graph(ebd)
de = Dense(100, activation='relu')(gr)
out = Dense(1, activation='sigmoid')(de)

mdl_pt = Model(input=inp, output=out)

mdl_pt.compile(Adam(), loss='binary_crossentropy', metrics=['accuracy'])

mdl_pt.fit(x_train_w, label_train, batch_size=32, nb_epoch=3,
           validation_data=(x_test_w, label_test))

### Dropout somewhere in between

In [ ]:
inp = Input(shape=(500,))
ebd = Embedding(input_dim=emb.shape[0], output_dim=emb.shape[1], input_length=500, 
                dropout=0.2, weights=[emb], trainable=False)(inp)
dr = Dropout(0.3)(ebd) 
gr = graph(dr)
dr2 = Dropout(0.3)(gr)
de = Dense(100, activation='relu')(dr2)
dr3 = Dropout(0.3)(de)
out = Dense(1, activation='sigmoid')(dr3)

mdl_pt = Model(input=inp, output=out)

mdl_pt.layers[1].trainable = True

mdl_pt.compile(Adam(), loss='binary_crossentropy', metrics=['accuracy'])

mdl_pt.fit(x_train_w, label_train, batch_size=32, nb_epoch=3,
           validation_data=(x_test_w, label_test))

### Add more dropout by the end

In [ ]:
inp = Input(shape=(500,))
ebd = Embedding(input_dim=emb.shape[0], output_dim=emb.shape[1], input_length=500, 
                dropout=0.2, weights=[emb], trainable=False)(inp)
dr = Dropout(0.3)(ebd) 
gr = graph(dr)
dr2 = Dropout(0.3)(gr)
de = Dense(100, activation='relu')(dr2)
dr3 = Dropout(0.5)(de)
out = Dense(1, activation='sigmoid')(dr3)

mdl_pt = Model(input=inp, output=out)

mdl_pt.layers[1].trainable = True

mdl_pt.compile(Adam(), loss='binary_crossentropy', metrics=['accuracy'])

mdl_pt.fit(x_train_w, label_train, batch_size=32, nb_epoch=3,
           validation_data=(x_test_w, label_test))

### Remove dropout from embedding layer

In [ ]:
inp = Input(shape=(500,))
ebd = Embedding(input_dim=emb.shape[0], output_dim=emb.shape[1], input_length=500, 
                weights=[emb], trainable=False)(inp)
dr = Dropout(0.3)(ebd) 
gr = graph(dr)
dr2 = Dropout(0.3)(gr)
de = Dense(100, activation='relu')(dr2)
dr3 = Dropout(0.3)(de)
out = Dense(1, activation='sigmoid')(dr3)

mdl_pt = Model(input=inp, output=out)

mdl_pt.layers[1].trainable = True

mdl_pt.compile(Adam(), loss='binary_crossentropy', metrics=['accuracy'])

mdl_pt.fit(x_train_w, label_train, batch_size=32, nb_epoch=3,
           validation_data=(x_test_w, label_test))

### Now add more dropout

In [ ]:
inp = Input(shape=(500,))
ebd = Embedding(input_dim=emb.shape[0], output_dim=emb.shape[1], input_length=500, 
                weights=[emb], trainable=False)(inp)
dr = Dropout(0.5)(ebd) 
gr = graph(dr)
dr2 = Dropout(0.5)(gr)
de = Dense(100, activation='relu')(dr2)
dr3 = Dropout(0.5)(de)
out = Dense(1, activation='sigmoid')(dr3)

mdl_pt = Model(input=inp, output=out)

mdl_pt.layers[1].trainable = True

mdl_pt.compile(Adam(), loss='binary_crossentropy', metrics=['accuracy'])

mdl_pt.fit(x_train_w, label_train, batch_size=32, nb_epoch=3,
           validation_data=(x_test_w, label_test))

### Add even more dropout to the last layer

In [ ]:
inp = Input(shape=(500,))
ebd = Embedding(input_dim=emb.shape[0], output_dim=emb.shape[1], input_length=500, 
                weights=[emb], trainable=False)(inp)
dr = Dropout(0.5)(ebd) 
gr = graph(dr)
dr2 = Dropout(0.5)(gr)
de = Dense(100, activation='relu')(dr2)
dr3 = Dropout(0.7)(de)
out = Dense(1, activation='sigmoid')(dr3)

mdl_pt = Model(input=inp, output=out)

mdl_pt.layers[1].trainable = True

mdl_pt.compile(Adam(), loss='binary_crossentropy', metrics=['accuracy'])

mdl_pt.fit(x_train_w, label_train, batch_size=32, nb_epoch=3,
           validation_data=(x_test_w, label_test))

### Add Batch Normalization

In [ ]:
inp = Input(shape=(500,))
ebd = Embedding(input_dim=emb.shape[0], output_dim=emb.shape[1], input_length=500, 
                weights=[emb], trainable=False)(inp)
bn = BatchNormalization()(ebd)
dr = Dropout(0.5)(bn) 
gr = graph(dr)
bn2 = BatchNormalization()(gr)
dr2 = Dropout(0.5)(bn2)
de = Dense(100, activation='relu')(dr2)
bn3 = BatchNormalization()(de)
dr3 = Dropout(0.5)(bn3)
out = Dense(1, activation='sigmoid')(dr3)

mdl_pt = Model(input=inp, output=out)

mdl_pt.layers[1].trainable = True

mdl_pt.compile(Adam(), loss='binary_crossentropy', metrics=['accuracy'])

mdl_pt.fit(x_train_w, label_train, batch_size=32, nb_epoch=3,
           validation_data=(x_test_w, label_test))

### More dropout

In [28]:
inp2 = Input(shape=(5000,50))
convs = []
for i in range(3,6):
    x = Convolution1D(64, i, border_mode='same', activation='relu')(inp2)
    x = MaxPooling1D()(x)
    x = Flatten()(x)
    convs.append(x) 
out2 = Merge(mode="concat")(convs)
graph = Model(input=inp2, output=out2)

inp = Input(shape=(500,))
ebd = Embedding(input_dim=emb.shape[0], output_dim=emb.shape[1], input_length=500, 
                weights=[emb], trainable=True)(inp)
# bn = BatchNormalization()(ebd)
dr = Dropout(0.2)(ebd) 
gr = graph(dr)
# bn2 = BatchNormalization()(gr)
dr2 = Dropout(0.2)(gr)
de = Dense(100, activation='relu')(dr2)
# bn3 = BatchNormalization()(de)
dr3 = Dropout(0.3)(de)
out = Dense(1, activation='sigmoid')(dr3)

mdl_pt = Model(input=inp, output=out)

mdl_pt.compile(Adam(0.01), loss='binary_crossentropy', metrics=['accuracy'])

mdl_pt.fit(x_train_w, label_train, batch_size=32, nb_epoch=3,
           validation_data=(x_test_w, label_test))

Train on 25000 samples, validate on 25000 samples
Epoch 1/3
25000/25000 [==============================] - 28s - loss: 0.4282 - acc: 0.8107 - val_loss: 0.3719 - val_acc: 0.8378
Epoch 2/3
25000/25000 [==============================] - 28s - loss: 0.3539 - acc: 0.8631 - val_loss: 0.3478 - val_acc: 0.8490
Epoch 3/3
25000/25000 [==============================] - 28s - loss: 0.3291 - acc: 0.8732 - val_loss: 0.3434 - val_acc: 0.8702


In [29]:
mdl_pt.fit(x_train_w, label_train, batch_size=32, nb_epoch=3,
           validation_data=(x_test_w, label_test))

Train on 25000 samples, validate on 25000 samples
Epoch 1/3
25000/25000 [==============================] - 28s - loss: 0.3402 - acc: 0.8621 - val_loss: 0.3851 - val_acc: 0.8691
Epoch 2/3
25000/25000 [==============================] - 28s - loss: 0.3921 - acc: 0.8386 - val_loss: 0.4317 - val_acc: 0.8310
Epoch 3/3
25000/25000 [==============================] - 28s - loss: 0.3845 - acc: 0.8423 - val_loss: 0.5495 - val_acc: 0.8563
